# Guessing Bits: Improved Lattice Attacks on(EC)DSA with Nonce Leakage (Sun et al., 2021) LeetArxiv Implementation

This coding guide is best followed alongside this [LeetArxiv article link](https://leetarxiv.substack.com/p/guessing-bits-improved-lattice-attacks).



The paper, *Guessing Bits: Improved Lattice Attacks on(EC)DSA with Nonce Leakage* , improves on the (Albrecht & Heninger, 2020) lattice-based HNP attack by:


1. Guessing some secret key bits to increase attack success probability.

2. Decomposing the secret key into batches to recover parts of the secret. ie. it’s no longer an ‘all-or-nothing’ approach.

## Install Libraries
We need:
1. **fpylll** for lattice construction.
2. **cysignals** to interact with fpylll's C++ code

In [1]:
#Install fpylll for lattices
!pip install fpylll
!pip install cysignals

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 6.6 MB/s eta 0:00:00


## Recentering and Projected Lattice

This section constructs a lattice with both the 'recentering trick' and 'projection trick' as described in section 2.5 and 2.6 of (Sun et al., 2021) [here](https://leetarxiv.substack.com/p/guessing-bits-improved-lattice-attacks)

We recover the secret key $\alpha$ from the second last row element using the equation:

$\alpha = t_0^{-1} \cdot (a_0 \cdot (\text{target} / (2^{l+1})) \pmod{q}$

In [244]:
import math
import random
from random import randint
from fpylll import FPLLL
from fpylll import *
from fpylll.util import gaussian_heuristic
from decimal import Decimal
from time import time

#Function to eliminate secret key
def EliminateSecretKeyTransform(prime, multipliers, observations):
  if len(multipliers) != len(observations):
    raise ValueError(f'Expected number of observations to equal number of multipliers, but got {len(observations)} and {len(multipliers)}.')
  m = len(multipliers)
  t0_inv = pow(multipliers[0], -1, prime)
  multipliers_new = []
  observations_new = []

  for i in range(1, m):
    ti_prime = (multipliers[i] * t0_inv) % prime
    ai_prime = (observations[i] - ti_prime * observations[0]) % prime
    multipliers_new.append(ti_prime)
    observations_new.append(ai_prime)

  return multipliers_new, observations_new, multipliers[0], observations[0]

"""
prime = q
multipliers = t
observations = v
noOfLeakedBits = l
latticeDimension = d
"""
def HNP_EliminateSecretKey_Projection(prime, multipliers, observations, errorBound, noOfLeakedBits):
  latticeDimension = len(multipliers) + 1
  #Eliminate the secret key
  multipliers_new, observations_new, t0, a0 = EliminateSecretKeyTransform(prime, multipliers, observations)
  assert(latticeDimension == len(multipliers_new)+2)
  #Build Projected lattice
  lattice = IntegerMatrix(latticeDimension, latticeDimension)
  #Set identity to prime * 2^(noOfLeakedBits + 1)
  for i in range(len(multipliers_new)):
    lattice[i, i] = int(prime) * (2 ** (noOfLeakedBits + 1))
  #Stack (multipliers and observations) * 2^(noOfLeakedBits + 1)
  for j in range(len(multipliers_new)):
    lattice[len(multipliers_new), j] = multipliers_new[j] * (2 ** (noOfLeakedBits + 1))
    lattice[len(multipliers_new)+1, j] = observations_new[j] * (2 ** (noOfLeakedBits + 1))
  #We replace B/p with 1 * (2 ** (noOfLeakedBits + 1)
  lattice[len(multipliers_new), len(multipliers_new)] = int(1) * (2 ** (noOfLeakedBits + 1))
  #Set error bound
  lattice[len(multipliers_new)+1, len(multipliers_new)+1] = errorBound
  #Perform LLL reduction
  #print(lattice)
  LLL.reduction(lattice)
  t0_inv = pow(t0, -1, prime)
  leakInverse = pow(2 ** (noOfLeakedBits + 1), -1, prime)
  #Search in lattice rows for secret key
  secretKey = prime
  for i in range(lattice.nrows):
    row = [lattice[i, j] for j in range(lattice.ncols)]
    #Target changes
    targetValue = row[-2] // (2 ** (noOfLeakedBits + 1))
    if row[-1] == -errorBound:
      alpha = (t0_inv * (a0 + targetValue)) % prime
      print(f"{alpha} : {math.log(alpha,2)}")
      if(alpha < secretKey):
        secretKey = alpha
    if row[-1] == errorBound:
      alpha = (t0_inv * (a0 - targetValue)) % prime
      print(f"{alpha} : {math.log(alpha,2)}")
      if(alpha < secretKey):
        secretKey = alpha
  return secretKey



In [3]:
def TestHNP_EliminateSecretKey(secretKey):
    #random.seed(9665096)
    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate = 65
    errorMaxExponent = 200
    multiplierMaxExponent = 230
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = math.ceil(math.log(q, 2)) - errorMaxExponent
    multiplierBound= 2**multiplierMaxExponent
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = randint(1, multiplierBound);s = randint(0, errorBound)
        c = (r*secretKey + s) % q
        #print(i, " : ", math.log(abs(s - halfErrorBound), 2))
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)

    #Predict
    prediction = HNP_EliminateSecretKey_Projection(q, randomMultipliers, results, errorBound,noOfLeakedBits)
    print(f"Target: {secretKey}")
    print(f"Prediction: {prediction}: {math.log(prediction,2)}")

if __name__ == "__main__":
  randomValue = (randint(2**134, 2**135))
  secretKey = randomValue
  TestHNP_EliminateSecretKey(secretKey)

28807859445984240125177079322705400269081 : 134.40358626121989
Target: 28807859445984240125177079322705400269081
Prediction: 28807859445984240125177079322705400269081: 134.40358626121989


## Increasing Lattice Volume

Here’s the paper’s thesis: we don’t need to find the entire secret key at once, we can split it and search in batches.

More formally, we can modify the lattice to increase its volume while the error vector remains constant.

This is Section 3.0 of the [LeetArxiv article](https://leetarxiv.substack.com/p/guessing-bits-improved-lattice-attacks).

In [186]:
import math
import random
from random import randint
from fpylll import FPLLL
from fpylll import *
from fpylll.util import gaussian_heuristic
from decimal import Decimal
from time import time

#Function to eliminate secret key
def EliminateSecretKeyTransform_Embedding(prime, multipliers, observations, embeddingFactor):
  if len(multipliers) != len(observations):
    raise ValueError(f'Expected number of observations to equal number of multipliers, but got {len(observations)} and {len(multipliers)}.')
  m = len(multipliers)
  t0_inv = pow(multipliers[0], -1, prime)
  multipliers_new = []
  observations_new = []

  for i in range(1, m):
    ti_prime = (multipliers[i] * embeddingFactor * t0_inv) % prime
    ai_prime = (observations[i] - ti_prime * observations[0]) % prime
    multipliers_new.append(ti_prime)
    observations_new.append(ai_prime)

  return multipliers_new, observations_new, multipliers[0], observations[0]

"""
prime = q
multipliers = t
observations = v
noOfLeakedBits = l
latticeDimension = d
"""
def HNP_EliminateSecretKey_Volume(prime, multipliers, observations, errorBound, noOfLeakedBits, volume):
  latticeDimension = len(multipliers) + 1
  #Eliminate the secret key
  multipliers_new, observations_new, t0, a0 = EliminateSecretKeyTransform(prime, multipliers, observations)
  assert(latticeDimension == len(multipliers_new)+2)
  #Build Projected lattice
  lattice = IntegerMatrix(latticeDimension, latticeDimension)
  #Set identity to prime * 2^(noOfLeakedBits + 1)
  for i in range(len(multipliers_new)):
    lattice[i, i] = int(prime) * (2 ** (noOfLeakedBits + 1))
  #Stack (multipliers and observations) * 2^(noOfLeakedBits + 1)
  for j in range(len(multipliers_new)):
    lattice[len(multipliers_new), j] = multipliers_new[j] * (2 ** (noOfLeakedBits + 1))
    lattice[len(multipliers_new)+1, j] = observations_new[j] * (2 ** (noOfLeakedBits + 1))
  #We replace B/p with 1 * (2 ** (noOfLeakedBits + 1 + volume)
  lattice[len(multipliers_new), len(multipliers_new)] = int(1) * (2 ** (noOfLeakedBits + 1 + volume) )
  #Set error bound
  lattice[len(multipliers_new)+1, len(multipliers_new)+1] = errorBound
  #Perform LLL reduction
  #print(lattice)
  LLL.reduction(lattice)
  t0_inv = pow(t0, -1, prime)
  leakInverse = pow(2 ** (noOfLeakedBits + 1), -1, prime)
  #Search in lattice rows for secret key
  secretKey = prime
  for i in range(lattice.nrows):
    row = [lattice[i, j] for j in range(lattice.ncols)]
    #Target changes
    targetValue = row[-2] // (2 ** (noOfLeakedBits + 1 + volume))
    if row[-1] == -errorBound:
      alpha = (t0_inv * (a0 + targetValue)) % prime
      alpha >>= volume
      print(f"{alpha} : {math.log(alpha,2)}")
      if(alpha < secretKey):
        secretKey = alpha
    if row[-1] == errorBound:
      alpha = (t0_inv * (a0 - targetValue)) % prime
      alpha >>= volume
      print(f"{alpha} : {math.log(alpha,2)}")
      if(alpha < secretKey):
        secretKey = alpha
  return secretKey




In [242]:
def TestHNP_EliminateSecretKey(secretKey, volume):

    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate =200
    errorMaxExponent = 233
    multiplierMaxExponent = 125
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = math.ceil(math.log(q, 2)) - errorMaxExponent
    multiplierBound= 2**multiplierMaxExponent
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = randint(1, multiplierBound);s = randint(0, errorBound)
        c = (r*secretKey + s) % q
        #print(i, " : ", math.log(abs(s - halfErrorBound), 2))
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)

    #Predict
    prediction = HNP_EliminateSecretKey_Volume(q, randomMultipliers, results, errorBound,noOfLeakedBits,volume)
    secretKey >>= volume
    print(f"Target: {secretKey}\n")
    print(f"Prediction: {prediction}: {math.log(prediction,2)}")

if __name__ == "__main__":
  random.seed(966508)
  randomValue = (randint(2**134, 2**135))
  volume = 100
  secretKey = randomValue << volume

  TestHNP_EliminateSecretKey(secretKey, volume)

35555136710425978440849335508674701825633 : 134.7071817989767
125142003583487477290534086958546341249643 : 136.5226179979561
64635119576618207962251063354638951551476 : 135.569442063689
109754585990801268343934655562937432698765 : 136.3333331130396
26725448859760884990338081943716185051639 : 134.2953379740226
24455342603754846888228127795226289576664 : 134.1672734715493
136119191219376580177292857294413016453759 : 136.6439223747488
36912720599033670937059335695346510650074 : 134.76124186876177
22178994402017465419862453323955942440441 : 134.02631775052924
109304095029702112398758843449729376823312 : 136.32739934233746
36876469719275662541226523254088298407842 : 134.7598243449094
18430665876522343314867103518693024734735 : 133.75923199014204
71370283906269183703604307041540266972223 : 135.7124473069356
37979945093139330364150749661431052485704 : 134.80236161530755
Target: 35555136710425978440849335508674701825533

Prediction: 18430665876522343314867103518693024734735: 133.75923199014204

# Bonus Content

## Double Recenter trick
1. Idk why but you can combine the 'increase exponent by 1' and half the error by 2 trick. So you effectively lower your bound by 2 bits lol

So this works lol

In [561]:
def TestHNP_EliminateSecretKey(secretKey, volume):

    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate =20
    errorMaxExponent = 235
    multiplierMaxExponent = 15
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = 2#math.ceil(math.log(q, 2)) - errorMaxExponent
    multiplierBound= 2**multiplierMaxExponent
    halfErrorBound = errorBound // 2
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = randint(1, multiplierBound);s = randint(0, errorBound)
        c = (r*secretKey + s) % q
        c = (c - halfErrorBound) % q
        #print(i, " : ", math.log(abs(s - halfErrorBound), 2))
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)

    #Predict
    prediction = HNP_EliminateSecretKey_Volume(q, randomMultipliers, results, errorBound//2,noOfLeakedBits,volume)
    secretKey >>= volume
    print(f"Target: {secretKey}\n")
    print(f"Prediction: {prediction}: {math.log(prediction,2)}")

if __name__ == "__main__":
  #random.seed(966508)
  randomValue = (randint(2**134, 2**135))
  volume = 100
  secretKey = randomValue << volume

  TestHNP_EliminateSecretKey(secretKey, volume)

22291487727851853337550974326510503519307 : 134.03361670038151
30497054215174222190844106770375177352710 : 134.48579369146822
153054660692149826976245704472126083987527 : 136.81309886765496
110140299236072209199256900917794152982923 : 136.33839432359298
Target: 22291837396501545122703789902235742278948

Prediction: 22291487727851853337550974326510503519307: 134.03361670038151


In [279]:
def TestHNP_EliminateSecretKey(secretKey):
    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate = 250
    errorMaxExponent = 234
    multiplierMaxExponent = 105
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = math.ceil(math.log(q, 2)) - errorMaxExponent
    multiplierBound= 2**multiplierMaxExponent
    halfErrorBound = errorBound // 2
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = randint(1, multiplierBound);s = randint(0, errorBound)
        c = (r*secretKey + s) % q
        c = (c - halfErrorBound) % q
        #print(i, " : ", math.log(abs(s - halfErrorBound), 2))
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)

    #Predict
    prediction = HNP_EliminateSecretKey_Projection(q, randomMultipliers, results, errorBound//2,noOfLeakedBits)
    print(f"Target: {secretKey}")
    print(f"Prediction: {prediction}: {math.log(prediction,2)}")

if __name__ == "__main__":
  random.seed(96650946)
  randomValue = (randint(2**134, 2**135))
  secretKey = randomValue
  TestHNP_EliminateSecretKey(secretKey)

15820724342386000368936630743620391990982 : 133.5389394497455
80555538229247935731737672827444724439847232743224236177533217431669614 : 235.54495042038562
16542000166056063789090203552406527092862481312512080423317540738355559 : 233.26110032977968
52231216112354991125664816531706114458916819008413389003575161852595097 : 234.91987893684296
68331749277184343107325293575536678184740850566009565886621113983870406 : 235.3075227020644
56678357323745640364837844513893204771406955501703920138193583149444994 : 235.03776458817603
42461840194582606241930551794643915927576473554304853325891321601726790 : 234.62113353766554
121467756034851918003856758280300737037527515984969785825487310414614173 : 236.1374681341073
131790704492794185491571407050081966516389283195394961748288739135974319 : 236.25514335427525
59733349362766116938074150388616169675575530815054070558698922647688586 : 235.1135032608241
107514814548291109111095471684846275242046163616792702577163440913926422 : 235.96143020056437
11991731